In [26]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms, models
from torch import nn, optim
import os

##### elements of both set are consists of tuples: 
image in index 0, label in index 1

In [3]:
# train = True for training set
trainset = datasets.MNIST("trainset", download=True, train=True, transform=None)

100%|██████████| 9912422/9912422 [00:01<00:00, 5712839.76it/s]


Extracting trainset\MNIST\raw\train-images-idx3-ubyte.gz to trainset\MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 28569739.11it/s]


Extracting trainset\MNIST\raw\train-labels-idx1-ubyte.gz to trainset\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 2396638.62it/s]


Extracting trainset\MNIST\raw\t10k-images-idx3-ubyte.gz to trainset\MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 4558633.35it/s]

Extracting trainset\MNIST\raw\t10k-labels-idx1-ubyte.gz to trainset\MNIST\raw



In [32]:
# train = False for test set
testset = datasets.MNIST("testset", download=True, train=False, transform=None)

In [54]:
# The number of mnist image files and size of each image
print(
    f"Length of trainset: {len(trainset)}, testset: {len(testset)}\n"
    f"Shape of trainset: {np.array(trainset[0][0]).shape}"
)

Length of trainset: 60000, testset: 10000
Shape of trainset: (28, 28)


In [60]:
trainset[0][0]

#### Define Functions for Training

1. Euclidean distance

In [48]:
def pixel_euclidean_distance(img1, img2):
    return np.sqrt(np.sum(np.square(img1 - img2)))

2. KNN Classification

In [62]:
def kNearestNeighbour(test_image, train_images, k):
    distances = []

    # Calculate the distance between the test image and every image in the training set
    for img in train_images:
        distances.append(pixel_euclidean_distance(test_image, np.array(img[0]).flatten()))

    # Sort the distances and get the indices of k smallest distances
    k_indices = np.argsort(distances)[:k]

    # Retrieve the labels of the k nearest neighbors
    k_nearest_labels = [train_images[i][1] for i in k_indices]

    return k_indices, k_nearest_labels

#### Single Image Test

In [77]:
def predict_single_image(test_image, test_image_label, train_images_set, k):
    # Get the indices and labels of the k nearest neighbors
    nearest_indices, nearest_labels = kNearestNeighbour(test_image, train_images_set, k)

    # find frequency of each value
    result_indices, counts = np.unique(nearest_labels, return_counts=True)
    # display each value with highest frequency
    result_nearest_labels = result_indices[counts == counts.max()]

    # Check if the test label is among the labels of the nearest neighbors
    return f"Actual label: {test_image_label}, Result: {result_nearest_labels}, Candidates: {nearest_labels}"
    # Return the most common label

In [78]:
single_test_image = np.array(testset[0][0]).flatten()
single_test_image_label = testset[0][1]

predict_single_image(single_test_image, single_test_image_label, trainset, 5)

'Actual label: 7, Result: [7], Candidates: [7, 7, 7, 1, 7]'

In [ ]:
def test_knn_accuracy(test_images, test_labels, train_images, train_labels, k):
    correct_cases = 0
    total_cases = len(test_images)

    for i in range(total_cases):
        single_test_image = test_images[i]
        single_test_image_label = test_labels[i]

        # Get the nearest neighbors
        _, result_labels = kNearestNeighbour(
            single_test_image, train_images, train_labels, k
        )

        # Find the most frequent label among the nearest neighbors
        result_indices, counts = np.unique(result_labels, return_counts=True)
        result_nearest_labels = result_indices[counts == counts.max()]

        # Check if the test label is among the labels of the nearest neighbors
        if single_test_image_label in result_nearest_labels:
            correct_cases += 1

    # Calculate accuracy
    accuracy = correct_cases / total_cases
    return accuracy